<a href="https://colab.research.google.com/github/gagabrie/COMP-593/blob/main/WEEK_3_WORKING_WITH_DATABASES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAB 3: WORKING WITH DATABASES


## Introduction

During this lab, we will experiment with using a few different libararies and modules to connect to and interact with an SQLite database. After creating and configuring our testing database, we will populate it with Fake data and run some queries to see what kind of information we can retreive. For context, we will be creating RSSN, the "Really Simple Social Network".

## Creating a SQLite Database

We will be utilizing the `sqlite3` library to create our database file, as it offers the convienience of creating our database file automatically in the event that it cannot find an existing file at the path specified.

**Before you run the below code:**
On the left of the notebook, select the file folder icon. Then, after you run the code block, you should be able to observe the creation of the database file. If it doesn't appear, try clicking the "Refresh" button above the list of folders.

In [29]:
import sqlite3

#When we retreive a Connection object, a new database will be created for us if it doesn't already exist. 
myConnection = sqlite3.connect('social_network.db')
print(sqlite3.version)

2.6.0


## Creating a Table


We'll use the below code to create a table called `people` within our database. 

In [30]:
import sqlite3

#Retreive the Connection object
myConnection = sqlite3.connect('social_network.db')

#Once we have a Connection object, we can generate a Cursor object, and use that to run our SQL Queries
myCursor = myConnection.cursor()

#Let's define the SQL Query we will use to create our first table:
createPeopleTable = """ CREATE TABLE IF NOT EXISTS people (
                          id integer PRIMARY KEY,
                          name text NOT NULL,
                          email text NOT NULL,
                          address text NOT NULL,
                          city text NOT NULL,
                          province text NOT NULL,
                          country text NOT NULL,
                          phone text,
                          bio text,
                          dob date NOT NULL,
                          heatmap integer,
                          created_at datetime NOT NULL,
                          updated_at datetime NOT NULL,
                          ipv4 text
                        );"""

#Now that we have the string to create our table,
#Cursor objects have an execute() method which will accept an SQL string and perform the operations described.

myCursor.execute(createPeopleTable)

#We can confirm if our table was created successfully by running the following SQL Query
#pragma_table_info is an internal SQLite function that will retun information about a table
myCursor.execute("SELECT group_concat(name, ', ') FROM pragma_table_info('people')")
print(myCursor.fetchone())

#We use to the commit() method on the database Connection object to persist our changes
myConnection.commit()

#It is always a good idea to close a connection when it will no longer be used
myConnection.close()


('id, name, email, address, city, province, country, phone, bio, dob, heatmap, created_at, updated_at, ipv4',)



If you received a tuple containing the names of the columns, awesome! We have successfully created our database table.
```
('id, name, email, address, city, province, country, phone, bio, created_at, updated_at',)
```

Run the below code block to add our first entry.

In [31]:
import sqlite3
from pprint import pprint #Outputs data in a slightly easier to read format
from datetime import datetime #For generating dates and times

#Retreive the Connection object
myConnection = sqlite3.connect('social_network.db')

#Once we have a Connection object, we can generate a Cursor object, and use that to run our SQL Queries
myCursor = myConnection.cursor()

#Let's define the SQL Query we will use to create our first entry:
addPersonQuery = """INSERT INTO people (name, 
                      email, 
                      address, 
                      city, 
                      province, 
                      country, 
                      phone, 
                      bio,
                      dob,
                      heatmap,
                      created_at, 
                      updated_at, 
                      ipv4)
                  VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);"""

"""
The ?'s are placeholders that we can fill in when we use the execute() method.
This is really handy for code reuse, as we can pass those values are variables in a tuple
Instead of hard coding them into the statement.
"""

myPerson = ("John Doe", 
            "johndoe@anon.null", 
            "123 Fake St.", 
            "Fakesville", 
            "Fakesdom", 
            "Fakopolis", 
            None, 
            None,
            "1967-09-12",
            0,
            datetime.now(), 
            datetime.now(),
            None)

myCursor.execute(addPersonQuery, myPerson)

#We can confirm if our table was created successfully by running the following SQL Query
#pragma_table_info is an internal SQLite function that will retun information about a table
myCursor.execute("SELECT * FROM people")
pprint(myCursor.fetchall())

# If you run this code block a few times, you will see that you only have 1 entry,
# If you uncomment the below lines and run the block a few more times, you will begin to see multiple entries.
# myConnection.commit()
# myConnection.close()

[(1,
  'John Doe',
  'johndoe@anon.null',
  '123 Fake St.',
  'Fakesville',
  'Fakesdom',
  'Fakopolis',
  None,
  None,
  '1967-09-12',
  0,
  '2022-02-24 03:34:30.272541',
  '2022-02-24 03:34:30.272546',
  None)]


# Lab Submission

We're going to build our experience with working with Libraries and examining documentation by populating our 'People' table with data provided to us by the `Faker` library. `Faker` is used to generate fake data and is very helpful for the rapid generation of databases for the purposes of testing. Run the two blocks below to install faker and get an idea of what it can do.

In [26]:
!pip install faker

In [32]:
from faker import Faker

fake = Faker()

for _ in range(10):
  print('{} || {}'.format( fake.name(), fake.job() ) )

Hannah Powell || Armed forces operational officer
Mindy Williams || Automotive engineer
Samantha Carr || Nature conservation officer
Michelle Smith || Print production planner
Andrea Burton || Psychotherapist, child
Aaron Fitzpatrick || Dispensing optician
Patrick Casey || Corporate investment banker
Daniel Smith || Building surveyor
Rebecca Gonzales || Risk manager
Evelyn Walsh || Government social research officer


Very cool! Faker has tons of `providers` that can all be used to populate fake data. The [list of providers](https://faker.readthedocs.io/en/stable/providers.html) in the Faker documentation will help you fill out the columns for our People table.

The goal of this script is to populate the people table with 1000 entries, with the following constraints:

1. The `heatmap` column must contain a random number between `999` and `2500`
2. The `created_at` and `updated_at` columns must use the `datetime` object (see examples above) 
2. Use `Faker` to generate all other fields.

*Hint: Each of the methods contained in the provider can be called directly from the base Faker object, for example, one can call the `file_name()` method from `faker.providers.file` by calling `Faker().file_name()`*

In [ ]:
fake.address

In [33]:
import sqlite3
from faker import Faker
from datetime import datetime #For generating dates and times
#Don't forget to import the module to generate random numbers!

#Retreive the Connection object
myConnection = sqlite3.connect('social_network.db')

#Once we have a Connection object, we can generate a Cursor object, and use that to run our SQL Queries
myCursor = myConnection.cursor()

#This is the same syntax as the above example:
addPersonQuery = """INSERT INTO people (name, 
                      email, 
                      address, 
                      city, 
                      province, 
                      country, 
                      phone, 
                      bio,
                      dob,
                      heatmap,
                      created_at, 
                      updated_at, 
                      ipv4)
                  VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);"""

fake = Faker()


for _ in range(1000):
  myPerson = (fake.name(), 
            fake.ascii_email(), 
            fake.address(), 
            fake.city(),
            fake.administrative_unit(), 
            fake.country_code(), 
            fake.phone_number(), 
            fake.text(),
            fake.date_between(),
            fake.random_int(min = 999,max = 2500),
            datetime.now(), 
            datetime.now(),
            fake.ipv4())
  myCursor.execute(addPersonQuery, myPerson)
myConnection.commit()

myCursor.execute("SELECT * FROM people")
pprint(myCursor.fetchall())

#Ideally, you will want to create a loop that will iterate 1000 times,
#Observe the example in the earlier code block showing the parameter tuple `myPerson`
#You can duplicate that inside your loop and replace those fields with calls to the appropriate faker provider method
#Execute your statement inside the loop,
#But, don't forget to commit and close your connection when you have finished!

Streaming output truncated to the last 5000 lines.
  '220.215.179.221'),
 (684,
  'Christopher Castro',
  'sbradley@gmail.com',
  '469 Taylor Drive\nJamestown, MA 29544',
  'Jonathanchester',
  'Idaho',
  'VE',
  '189.335.9918',
  'Clearly PM drug rate happy. Republican successful environment whole green. '
  'Lot today seat him hold.',
  '1996-09-06',
  1768,
  '2022-02-24 03:35:59.036502',
  '2022-02-24 03:35:59.036504',
  '115.171.162.254'),
 (685,
  'Stephen Brown',
  'lauriewong@adams-smith.com',
  '80866 Mcgee Mission\nLopezville, MO 40732',
  'Anthonyview',
  'Arkansas',
  'NR',
  '+1-146-979-5378',
  'Public our much interest coach. Natural pick newspaper move. Industry note '
  'strategy maybe. Poor recently later rock.\n'
  'Focus series consumer loss soon. Kid history surface wife sometimes '
  'everyone.',
  '2011-01-28',
  2068,
  '2022-02-24 03:35:59.037699',
  '2022-02-24 03:35:59.037700',
  '91.57.148.37'),
 (686,
  'Donald Cuevas Jr.',
  'phillip16@johnson-wright.com',

Finally, the last step,
Using the code block below, combined with what you have learned above, and the Lecture notes,
Craft a SQL Query that will return the `name` of no more than `20` `people` with a `heatmap` greater than `1500`

In [37]:
import sqlite3
import pprint
#Retreive the Connection object
myConnection = sqlite3.connect('social_network.db')

#Once we have a Connection object, we can generate a Cursor object, and use that to run our SQL Queries
myCursor = myConnection.cursor()

selectStatement = """SELECT name from people where heatmap > 1500  limit 20 """

myCursor.execute(selectStatement)
results = myCursor.fetchall()
pprint.pprint(results)

[('Jessica Davis',),
 ('Sean Reed',),
 ('Michael Hernandez',),
 ('Michael Bailey',),
 ('Brandon Clark',),
 ('Carol Harris',),
 ('Kenneth Williams',),
 ('Trevor Pham',),
 ('Joshua Turner',),
 ('Lisa Bennett',),
 ('John Martin',),
 ('Stacy Taylor',),
 ('Amy Sutton',),
 ('Jay Guerrero',),
 ('Pamela Green',),
 ('Melissa Allen',),
 ('Danielle Ayers',),
 ('Laura Gutierrez',),
 ('Bethany Hart',),
 ('Brittany Romero',)]


Your submission will contain, as usual, a link to your completed colab notebook, but in addition to that, you will download a copy of your social_network.db file and upload it to D2L. To download the file, right click it from the Files menu on the left of the Notebook.